In [ ]:
LANGCHAIN_TRACING_V2 = True
LANGCHAIN_ENDPOINT =
LANGCHAIN_API_KEY =
LANGCHAIN_PROJECT =
SERPAPI_API_KEY =
API_TOKEN = 
OPENAI_API_KEY =

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# 환경변수 설정은 되어 있다고 가정합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("프로젝트명 기입")

LangSmith 추적을 시작합니다.
[프로젝트명]
langchain-stock


In [3]:
import torch
torch.cuda.empty_cache()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.agents import Tool
from langchain.tools import tool
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from difflib import get_close_matches


class NewsCollector:
    @tool("get_latest_news")
    def get_latest_news(self) -> list:
        """야후 파이낸스에서 최신 뉴스를 가져옵니다."""
        try:
            response = requests.get(
                "https://finance.yahoo.com/news",
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            new_items=[]
            soup = BeautifulSoup(response.text, 'html.parser')
            articles1 = soup.find_all('h3', class_='clamp tw-line-clamp-3 sm:tw-line-clamp-2 yf-18q3fnf')
            articles2 = soup.find_all('h2',class_= 'tw-line-clamp-3 yf-16ne7ux')
            articles = articles1+articles2
            for article in articles:
                new_items.append(article.text)

            return new_items
        except Exception as e:
            return f"뉴스 수집 실패: {str(e)}"

class Exchange:
    def __init__(self):
        self.base_url = "https://finance.naver.com/marketindex/"
    
    def get_usd_exchange_rate(self) -> dict:
        """네이버 증권에서 미국 USD 환율 정보를 가져옵니다."""
        try:
            response = requests.get(self.base_url, headers={"User-Agent": "Mozilla/5.0"})
            response.raise_for_status()  # 요청이 성공했는지 확인
            soup = BeautifulSoup(response.text, "html.parser")
            
            # 미국 USD 환율 데이터 가져오기
            usd_data = soup.select_one("div.market1 ul.data_lst li.on")
            if not usd_data:
                return {"error": "환율 정보를 가져올 수 없습니다."}
            
            # 필요한 데이터 추출
            value = usd_data.select_one("span.value").get_text(strip=True)  # 환율 값
            change = usd_data.select_one("span.change").get_text(strip=True)  # 변동 값
            blind = usd_data.select("span.blind")[2].get_text(strip=True)  # 설명 (상승/하락)
            
            return {"value": float(value.replace(",", "")), "change": float(change.replace(",", "")), "blind": blind}
        except Exception as e:
            return {"error": f"환율 정보를 가져오는 중 오류 발생: {str(e)}"}

            
        
        
class StockTools:
    def get_stock_price(self, tool_input: str) -> dict:
        """주식 심볼을 입력받아 현재 주가 정보를 반환합니다."""
        try:
            symbol = str(tool_input).strip().upper()
            stock = yf.Ticker(symbol)
            info = stock.info
            history = stock.history(period='1d')
            
            if history.empty:
                return {"error": f"데이터를 찾을 수 없습니다: {symbol}"}
                
            return info
        except Exception as e:
            return {"error": f"주가 조회 실패: {str(e)}"}

    def get_market_indices(self, tool_input: str = "") -> dict:
        """주요 시장 지수 정보를 반환합니다. 상승/하락 이유를 분석합니다."""
        indices = {'^GSPC': 'S&P 500', '^DJI': 'Dow Jones', '^IXIC': 'NASDAQ'}
        results = {}
        
        for symbol, name in indices.items():
            try:
                stock = yf.Ticker(symbol)
                history = stock.history(period='1d')
                if not history.empty:
                    close_price=history['Close'].iloc[-1]
                    open_price=history['Open'].iloc[0]
                    change=((close_price-open_price)/open_price)*100
                    results[name] = {
                        "price": close_price,
                        "change": change,
                        "trend":"상승" if change >=0 else "하락"
                    }
            except Exception:
                results[name] = {"error": "데이터 조회 실패"}
                
        return results


class FinancialAdvisor:
    def __init__(self):
        # 기본 LLM 설정
        self.gpt = ChatOpenAI(
            model_name="gpt-4", 
            temperature=0.7,
            api_key=OPENAI_API_KEY
            )
        # Bllossom 모델 설정
        self.setup_bllossom()
        
        # 도구 설정
        self.news_collector = NewsCollector()
        self.stock_tools = StockTools()
        
        # 메모리 설정
        self.memory = ConversationBufferWindowMemory(
            k=5,
            memory_key="chat_history",
            return_messages=True
        )
        self.terms_df=pd.read_excel('terms_data.xlsx')
        
        # 도구 목록 설정
        self.tools = [
            Tool(
                name="News",
                func=self.news_collector.get_latest_news,
                description="최신 금융 뉴스를 가져옵니다"
            ),
            Tool(
                name="StockPrice",
                func=self.stock_tools.get_stock_price,
                description="특정 주식의 현재 가격 정보를 가져옵니다"
            ),
            Tool(
                name="MarketIndices",
                func=self.stock_tools.get_market_indices,
                description="주요 시장 지수 정보를 가져옵니다"
            )
        ]
        
        # 체인 설정
        self.setup_chains()
        
        
    #양자화
    def setup_bllossom(self):
        quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                                bnb_4bit_compute_dtype=torch.float16,  # 연산 데이터 유형
                                                bnb_4bit_use_double_quant=True,       # 더블 양자화 활성화
                                                bnb_4bit_quant_type="nf4")         # 양자화 유형
        model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map={"":0},
            quantization_config=quantization_config,
            pad_token_id=tokenizer.eos_token_id,
        )
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            
        )
        
        self.bllossom = HuggingFacePipeline(pipeline=pipe)

    def setup_chains(self):
        
        # 의도 분석 체인
        intent_prompt = PromptTemplate(
            input_variables=["query"],
            template="""다음 질문의 의도를 파악해서 가장 적절한 번호 하나만 알려주세요.

                        질문: {query}

                        1번: 주식이나 투자에 대한 일반적인 설명이 필요한 경우
                        2번: 특정 주식의 현재 가격을 알고 싶은 경우
                        3번: 전체 시장 상황이나 동향을 알고 싶은 경우
                        4번: 최신 뉴스나 소식을 알고 싶은 경우
                        5번: 환율 정보를 알고 싶은 경우

                        답변:"""
        )
        
        self.intent_chain = LLMChain(
            llm=self.bllossom,
            prompt=intent_prompt,
            memory=self.memory,
            verbose=True,
        )
  
        # 주식 분석 체인
        stock_prompt = PromptTemplate(
            input_variables=["stock_data","query"],
            template="""
            다음 주식 정보를 바탕으로 질문에 맞는 답변을 해주세요:{stock_data}
            
            질문:{query}
            답변:
            """
        )
        
        self.stock_chain = LLMChain(
            llm=self.bllossom,
            prompt=stock_prompt,
            memory=self.memory,
            verbose=True
        )
        
        # 시장 분석 체인
        market_prompt = PromptTemplate(
            input_variables=["indices", "news"],
            template="""
            다음 정보를 바탕으로 현재 시장 상황을 분석해서 한국말로 알려주세요:
            
            시장 지수:
            {indices}
            
            주요 뉴스:
            {news}
            """
        )
        
        self.market_chain = LLMChain(
            llm=self.gpt,
            prompt=market_prompt,
            memory=self.memory,
            verbose=True
        )
        
    def classify_intent(self, query: str) -> str:
        """사용자 입력의 의도를 분류합니다."""
        try:
            stock_keywords=[]
            # 용어 사전 정의
            stock_keywords=self.terms_df['용어'].tolist()+self.terms_df['연관어'].tolist()
                
             # 기본 키워드 정의
            keywords = {
                "1": ["설명", "뭔가요", "의미", "개념","뭐야","란?"]+stock_keywords,
                "2": ["주가", "얼마", "가격", "시세"],
                "3": ["시장", "동향", "상황", "지수", "추세"],
                "4": ["뉴스", "소식", "기사", "새로운"],
                "5": ["환율","원화","달러"]
            }

            
            query_lower = str(query).lower()
            for intent, word_list in keywords.items():
                word_list = [str(word) for word in word_list]
                if any(word in query_lower for word in word_list):
                    return intent
            
            # 키워드로 판단이 어려운 경우 Bllossom 모델 사용
            response = self.intent_chain.run(query)            
            
            # 응답에서 숫자만 추출
            import re
            numbers = re.findall(r'\d+', response)
            if numbers:
                intent = numbers[0]
                if intent in ["1", "2", "3", "4"]:
                    return intent
                    
            # 기본값 반환
            return "3"
            
        except Exception as e:
            print(f"의도 분류 중 오류 발생: {str(e)}")
            return "1"  # 오류 발생 시 기본값
    def find_term_in_excel(self,query:str) -> str:
        """Excel 파일에서 관련 용어를 찾아 설명을 반환합니다."""
        #쿼리에서 주요 키워드 추출
        stock_word=query.replace("이란?", "").replace("란?", "").replace("가 뭐야?", "").replace("이 뭐야?", "").strip()
        #용어 열,행에서 일치하는 항목 검색
        term_match=self.terms_df[self.terms_df['용어'].str.contains(stock_word,na=False)]
        related_match = self.terms_df[self.terms_df['연관어'].str.contains(stock_word, na=False)]
        #합치기
        self.matches = pd.concat([term_match, related_match]).drop_duplicates()
        self.matches=self.matches.drop_duplicates(keep='first')

        if not self.matches.empty:
            row=get_close_matches(stock_word,self.matches, n=1, cutoff=0.8)
            context=f"""
            용어:{row['용어']}
            연관어:{row['연관어']}
            설명:{row['설명']}
            """
            return context
        return None
    
    def process_query(self, query: str) -> str:
        try:
            # 의도 파악
            intent = self.classify_intent(query)
            
            if intent == "1":
                #용어검색
                term_info=self.find_term_in_excel(query)
                if term_info:
                    prompt = f"""다음 정보를 바탕으로 질문에 답변해주세요:
                    {term_info}
                    질문:{query}
                    답변:"""
                    return self.bllossom(prompt)
                else:
                    return self.bllossom(query)
            
            elif intent == "2":
                # 주식 가격 조회
                symbol_prompt = PromptTemplate(
                    input_variables=["query"],
                    template="""사용자가 미국 주식과 관련된 질문을 합니다. 질문에서 미국 주식 심볼(티커)을 추출하세요. 
                                심볼은 대문자 영어로 구성된 1~5글자이며, 심볼만 반환하고 다른 설명은 포함하지 마세요. 
                                만약 질문에서 심볼을 추출할 수 없다면 미국 s&p500 심볼이라도 추출하세요.
                                질문: {query}
                                답변: 심볼(영어)
                                
                                """
                )
                symbol_chain = LLMChain(llm=self.gpt, prompt=symbol_prompt)
                symbol = symbol_chain.run(query).strip().upper()
                
                # Tool 실행 방식 수정
                stock_tool = [tool for tool in self.tools if tool.name == "StockPrice"][0]
                stock_data = stock_tool.run(symbol)
                
                return self.stock_chain.run(stock_data=stock_data)
            
            elif intent == "3":
                # 시장 동향 분석
                market_tool = [tool for tool in self.tools if tool.name == "MarketIndices"][0]
                indices = market_tool.run("")
                
                news_tool = [tool for tool in self.tools if tool.name == "News"][0]
                news = news_tool.run("")
                
                analysis_message=self.analyze_market(indices,news)
                
                return analysis_message
            
            elif intent == "4":
                # 뉴스 요약
                news_tool = [tool for tool in self.tools if tool.name == "News"][0]
                news = news_tool.run("")
                return self.gpt.predict(f"다음 뉴스를 한국말로 요약해주세요: {news}")
            
            elif intent == "5":
                # Exchange 클래스 인스턴스 생성
                exchange = Exchange()
                exchange_data = exchange.get_usd_exchange_rate()
                
                # 오류 처리
                if "error" in exchange_data:
                    return exchange_data["error"]
                
                # LLM Prompt 설정
                exchange_prompt = PromptTemplate(
                    input_variables=['query', 'value', 'change', 'blind'],
                    template="""
                    사용자가 환율에 대해 물어보면 현재 환율 정보를 출력해주세요.
                    만약 원화 또는 달러를 계산해달라고 요청하면 환율에 따라 결과를 계산해주세요.
                    질문{query}
                    
                    
                    현재 환율 정보:
                    - USD 환율: {value}원
                    - 변동: {change}원 ({blind})
                    """
                )
                
                # 환율 계산 체인 실행
                exchange_chain = LLMChain(llm=self.bllossom, prompt=exchange_prompt)
                response = exchange_chain.run(
                    query=query,
                    value=exchange_data["value"],
                    change=exchange_data["change"],
                    blind=exchange_data["blind"]
                )
                
                return response

            else:
                return "죄송합니다. 질문을 이해하지 못했습니다."
                
        except Exception as e:
            return f"오류가 발생했습니다: {str(e)}"
    
    def analyze_market(self,indeices:dict,news:list) ->str:
        """시장 지수와 뉴스를 바탕으로 상승/하락 이유를 분석합니다."""
        try:
            analysis =[]
            for index , data in indeices.items():
                trend=data.get("trend","정보없음")
                price=data.get("price","정보없음")
                change=data.get("change","정보없음")
                related_news = self.gpt.predict(f"다음 뉴스를 한국말로 요약해주세요:{news}")
                
                analysis.append(
                    f"{index}({trend}:현재 가격은 {price}, 변동률은 {change:.2f}%입니다.")
            analysis.append(related_news)
            return "\n\n".join(analysis)
        except Exception:
            return f"시장 분석 중 오류가 발생했습니다"
        

    def chat(self):
        print("금융 어시스턴트입니다. 종료하려면 'quit'를 입력하세요.")
        while True:
            query = input("질문: ")
            if query.lower() == 'quit':
                break
                
            response = self.process_query(query)
            print(f"답변: {response}")

if __name__ == "__main__":
    advisor = FinancialAdvisor()
    advisor.chat()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

금융 어시스턴트입니다. 종료하려면 'quit'를 입력하세요.
답변: 
                    사용자가 환율에 대해 물어보면 현재 환율 정보를 출력해주세요.
                    만약 원화 또는 달러를 계산해달라고 요청하면 환율에 따라 결과를 계산해주세요.
                    질문54달러 원화로
                    
                    
                    현재 환율 정보:
                    - USD 환율: 1396.4원
                    - 변동: 1.1원 (하락)
                     원화로 계산: 1달러 = 1396.4원
                    달러로 계산: 1달러 = 1396.4원
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
                    ```                    
        

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/%EB%AF%B8%EA%B5%AD%20S&P500%20%EC%8B%AC%EB%B3%BC%EC%9D%84%20%EC%B6%94%EC%B6%9C%ED%95%A0%20%EC%88%98%20%EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4.?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=%EB%AF%B8%EA%B5%AD+S%26P500+%EC%8B%AC%EB%B3%BC%EC%9D%84+%EC%B6%94%EC%B6%9C%ED%95%A0+%EC%88%98+%EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4.&crumb=IRoDw%2FyRiiH
$미국 S&P500 심볼을 추출할 수 없습니다.: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")




> Entering new LLMChain chain...
답변: 오류가 발생했습니다: Missing some input keys: {'query'}


C:\Users\yesthan\AppData\Local\Temp\ipykernel_13948\3588216118.py:360: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.gpt.predict(f"다음 뉴스를 한국말로 요약해주세요: {news}")


답변: - 아다니 그린은 회사 창립자인 가우탐 아다니가 미국 FCPA 혐의에서 명시되지 않았다고 주장하며 주가가 상승했다.
- 일부 운전자들은 전기차의 이동 범위 제한과 비용 때문에 회의적이라는 의견을 내놓고 있다. 
- 노르스크 하이드로는 녹색 수소와 배터리 프로젝트를 중단했다.
- 120억 달러의 기후 기금이 희귀한 채권 발행을 준비하고 있다.
- OpenAI는 직원들에게 15억 달러 주식을 소프트뱅크에 판매할 수 있게 허용했다고 CNBC가 보도했다.
- 투자자들이 관세에 대한 우려를 털어내고 연방준비제도(Fed)의 회의록을 소화하면서 S&P 500과 다우는 최고치를 기록했다.
- 전 NFL 스타 롭 그론코스키는 애플 주식이 그의 최고의 투자였다고 주장했다.
- 트럼프의 관세 우려로 아시아 주식이 하락했으며, 엔화가 강세를 보였다.
- 베스트 바이는 소비자들이 가전제품과 전자제품을 구매하는 것을 뒷받침하지 못하며 소득 추정치를 크게 놓쳤다.
- 스탠다드 차타드 분석가들은 비트코인이 100,000달러로 급등하기 전에 하락할 것으로 예상하였다. 
- 비트코인 ETF는 비트코인이 94,000달러 아래로 떨어지면서 4억3천8백만 달러를 유출했다.
- 애플은 2024년 스마트폰 시장 반등에서 뒤떨어졌다고 IDC가 밝혔다.
- 크라우드스트라이크는 실망스러운 수익 전망 후 주가가 하락했다.
답변: S&P 500(상승:현재 가격은 6021.6298828125, 변동률은 0.36%입니다.

Dow Jones(상승:현재 가격은 44860.30859375, 변동률은 0.55%입니다.

NASDAQ(상승:현재 가격은 19174.30078125, 변동률은 0.34%입니다.

'VW는 신장에서의 사업을 철수하며 현지 공장과 테스트 트랙을 매각할 예정이라는 소식이 전해졌다. 고탐 아다니는 증권법 위반 혐의로 기소되며, 아다니 그린은 이에 대해 부인했다. 아다니는 미국 FCPA 기소에서 창업자의 이름이 언급되지 않았다고 주장하며 주식이 상승했다. 트럼프의 인선에 집중하며 선물은 약세를